In [1]:
import dash
from dash import Dash,html,dcc,Input,Output,callback
import dash_bootstrap_components as dbc
from dash import dash_table as dt
from dash.dependencies import Input, Output, State
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objs as go

In [ ]:
#intputs
PNi = dbc.InputGroup(
    [dbc.InputGroupText("Project Name"), dbc.Input(id='PN',placeholder="Project Name")],
    className="mb-3")
TRi = dbc.InputGroup(
    [dbc.InputGroupText("Top Reservoir"), dbc.Input(id='TR',value="2100",type="number"),dbc.InputGroupText("TVDMSL")],
    className="mb-3")
BRi = dbc.InputGroup(
    [dbc.InputGroupText("Base Reservoir"), dbc.Input(id='BR',value="2300",type="number"),dbc.InputGroupText("TVDMSL")],
    className="mb-3")
NTGi = dbc.InputGroup(
    [dbc.InputGroupText("NTG"), dbc.Input(id='NTG',value="0.88",type="number"),dbc.InputGroupText("Fraction")],
    className="mb-3")
WDi = dbc.InputGroup(
    [dbc.InputGroupText("Water depth"), dbc.Input(id='WD',value="150",type="number"),dbc.InputGroupText("m")],
    className="mb-3")
RRi = dbc.InputGroup(
    [dbc.InputGroupText("Reservoir Radius"), dbc.Input(id='RR',value="3500",type="number"),dbc.InputGroupText("m")],
    className="mb-3")
Di = dbc.InputGroup(
    [dbc.InputGroupText("Depletion"), dbc.Input(id='D',value="13",type="number"),dbc.InputGroupText("bar")],
    className="mb-3")
Ei = dbc.InputGroup(
    [dbc.InputGroupText("Youngs Modulus, E"), dbc.Input(id='E',value="11",type="number"),dbc.InputGroupText("GPa")],
    className="mb-3")
nui = dbc.InputGroup(
    [dbc.InputGroupText("Poissons Ratio, nu"), dbc.Input(id='nu',value="0.12",type="number")],
    className="mb-3")
ei = dbc.InputGroup(
    [dbc.InputGroupText("End of data range"), dbc.Input(id='e',value="8000",type="number"),dbc.InputGroupText("m")],
    className="mb-3")
intvi = dbc.InputGroup(
    [dbc.InputGroupText("Data interval"), dbc.Input(id='intv',value="100",type="number"),dbc.InputGroupText("m")],
    className="mb-3")

In [9]:
# function to calculate the variables
def calcvariables(TR,BR,NTG,WD,Ew,nuw,RR):
    # inputs
    TR = float(TR)
    BR =float(BR)
    NTG =float(NTG)
    WD =float(WD)
    Ew =float(Ew)
    nuw =float(nuw)
    RR =float(RR)

    # calculations
    RT = (float(BR)-float(TR))*float(NTG)
    OB = float(TR)-float(WD)
    K = float(Ew)/(3*(1-2*float(nuw)))
    C = OB/float(RR)
    M = (float(Ew)*(1-float(nuw)))/((1-(2*float(nuw)))*(1+float(nuw)))
    B = 1-(K/37)
    Cm = 1/M/1000*B
    
    # make table of the calculated variables
    data = {
        'Calculated variable':  ['Reservoir thickness','Overburden thickness','Bulk Modulus, K','C (depth/radius)','Uniaxial Compaction Modulus, M','Biot','Cm'],
        'Value': [RT,OB,K,C,M,B,Cm],
        'Units': ['m','m','GPa','fraction','GPa','','MPa^*1'],
    }
    df = pd.DataFrame(data)
    df['Value'] = pd.to_numeric(df['Value'], errors='coerce') # formatting the value column as numbers
    df.loc[df.index[:-1], 'Value'] = df['Value'][:-1].apply(lambda x: '{:.2f}'.format(x)) # formatting value column to 2 sig figs apart from the last value
    last_row_index = len(df) - 1
    last_cell_value = df.at[last_row_index, 'Value'] 
    df.at[last_row_index, 'Value'] = '{:.2e}'.format(float(last_cell_value)) # formating last value to be a scientific type number with 2 sig figs
    columns = [{'name': col, 'id': col} for col in df.columns] # creating column variable to output into a datatable in the dash app
    data = df.to_dict('records')

    return TR,BR,NTG,WD,Ew,nuw,RR,RT,OB,K,C,M,B,Cm,data,columns

In [10]:
# function to calculate subsidence and compaction
def geertsma(TR, BR, NTG, WD, Ew, nuw,RR, e,intv,D):
    # getting calculated variables using variables function
    TR, BR, NTG, WD, Ew, nuw,RR, RT,OB,K,C,M,B,Cm,data,columns = calcvariables(TR,BR,NTG,WD,Ew,nuw,RR)
    # set data limits
    e = float(e)
    intv=float(intv)
    D=float(D)
    # create dataframe for a range of data points defined by the water depth (first depth) and depth of interest (number of data points controlled by input intv)
    df_results = pd.DataFrame((np.arange(WD,e,intv)), columns=['Depth'])   
    df_results['Z (m)'] = ((df_results['Depth']) - WD)/TR   # Creating normalised depth
    
    #creating conditions to asign data to OB, UB or reservoir
    conditions = [
        (df_results['Depth'] < TR),
        (df_results['Depth'] >= TR) & (df_results['Depth'] <= BR),
        (df_results['Depth'] > BR)
        ]
    df_results['definition'] = np.select(conditions, ['Overburden','Reservoir','Underburden'])   # defining data points as either overburden, reservoir and underburden
    
    # Calculation of Uz (broken into smaller componants)
    Z = df_results['Z (m)']
    a = (Cm/2)*RT*D*B
    b = (C*(Z-1))/pow(1+C*C*(Z-1)*(Z-1),(1/2))
    c = ((3-(4*nuw))*C*(Z+1))/pow(1+C*C*(Z+1)*(Z+1),(1/2))
    d = (2*C*Z)/pow((1+C*C*(Z+1)*(Z+1)),(3/2))
    e_OB = 3-(4*nuw)+1   # +1 used if overburden
    e_UB = 3-(4*nuw)-1   # -1 used if underburden
    conditions = [
        (df_results['definition'] == 'Overburden'),
        (df_results['definition'] == 'Reservoir'),
        (df_results['definition'] == 'Underburden')
        ]
    Uz_OB = a*(b-c+d+e_OB)
    Uz_UB = a*(b-c+d+e_UB)
    df_results['Uz (m)'] = np.select(conditions, [Uz_OB,Uz_OB,Uz_UB])   # calculation of Uz
    df_results['Uz (cm)'] = df_results['Uz (m)'] * 100
    
    Uz_end = df_results['Uz (m)'].iloc[-1]    # Extracting the end Uz value
    df_results['Vertical displacement (m)'] = df_results['Uz (m)'] - Uz_end    # Calculation of vertical displacement
    df_results['Vertical displacement (cm)'] = df_results['Vertical displacement (m)']*100

    df_maxD = df_results['Vertical displacement (cm)'].idxmax()     #finding index of row with max displacement - corresponds to top reservoir
    df_next = df_maxD + 1                                               #creating the index of next row after max displacement row
    df_results_compaction = df_results.loc[df_maxD:df_next]         # new dataframe with data at the top and base reservoir

    Vd_OB = df_results_compaction['Vertical displacement (cm)'].iloc[0]     # extracting the vertical displacement just above res
    Vd_UB = df_results_compaction['Vertical displacement (cm)'].iloc[-1]    # extracting the vertical displacement just beow the res
    Compaction = Vd_OB - Vd_UB   #calculating reservoir compaction
    Compaction_est = RT*D*Cm*100 
    sb_sub = df_results.iloc[0,6]    # sea bed subsidence is the first displacement value calculated for seabed depth

    # calculating strains
    Res_strain = ((Compaction/10) / RT)*100
    OB_strain = ((sb_sub/10)/ OB)*100

    # formating numbers to 2 decimal places
    Compaction = "{:.2f}".format(Compaction)
    sb_sub ="{:.2f}".format(sb_sub)
    Res_strain = "{:.2f}".format(Res_strain)
    OB_strain = "{:.2f}".format(OB_strain)
    
    sub = f'Seabed subsidence: {sb_sub} cm'
    compaction = f'Reservoir compaction: {Compaction} cm'
    strain_res = f'Reservoir strain: {Res_strain} %'
    strain_ob = f'Overburden strain: {OB_strain} %'
    
    return df_results, sub,compaction, Compaction, sb_sub, strain_res,strain_ob

In [11]:
def graph(TR, BR, NTG, WD, Ew, nuw,RR, e,intv,D):
    # geting data using previous functions
    df_results,sub,compaction, Compaction, sb_sub,strain_res,strain_OB = geertsma(TR, BR, NTG, WD, Ew, nuw,RR, e,intv,D)
    
    # setting depth limits
    maxD = df_results['Vertical displacement (cm)'].max()
    minD = df_results['Vertical displacement (cm)'].min()
    
    # setting horizontal lines to plot for surfaces
    x_range = [(minD-2),(maxD+2)]
    Seabed_y = (WD,WD)
    Res_top_y = (TR,TR)
    Res_base_y = (BR, BR)

    depthdata = {
        'xrange':[(minD-2),(maxD+2)],
        'Seabed_y': [WD,WD],
        'Res_top_y': [TR,TR],
        'Res_base_y': [BR, BR]
    }

    depth_df = pd.DataFrame(depthdata)

    profile = go.Scatter( #plot subsidence profile
        x= df_results['Vertical displacement (cm)'],
        y= df_results['Depth'],
        mode='lines',
        name='Displacement profile',
        line=dict(color='black')
    )

    sb = go.Scatter(   #plot sea bed
        x= depth_df['xrange'],
        y= depth_df['Seabed_y'],
        mode='lines',
        name='Seabed',
        line=dict(color='steelblue')
    )

    rest = go.Scatter(   #plot top res
        x= depth_df['xrange'],
        y= depth_df['Res_top_y'],
        mode='lines',
        name='Reservoir Top',
        line=dict(color='gold')
    )

    resb = go.Scatter(   #plot base res
        x= depth_df['xrange'],
        y= depth_df['Res_base_y'],
        mode='lines',
        name='Reservoir Base',
        line=dict(color='darkorange')
    )

    layout = go.Layout(
        xaxis=dict(title='Vertical displacement (cm)',side='top', mirror=True, title_standoff=5,title_font=dict(size=12, family='arial'),zeroline=True, zerolinewidth=1, zerolinecolor='grey'),
        yaxis=dict(title='Depth (mTVDMSL)',autorange='reversed',title_font=dict(size=12, family='arial'), title_standoff=5,zeroline=True, zerolinewidth=1, zerolinecolor='grey',mirror=True),
        legend=dict(orientation='h', x=0.1, y=0.16,bordercolor='black',borderwidth=1),
        height=800,
        plot_bgcolor='white',
        template='simple_white' 
        
    )

    fig = go.Figure(data=[profile,sb,rest,resb], layout=layout)

    return fig

In [44]:
# create dash app using functions above

app = dash.Dash(external_stylesheets=[dbc.themes.CERULEAN])

# setting layout and including all the componants

app.layout = dbc.Card([html.Div(
    [dbc.Row([
        html.H1('Subsidence Calculator',style={'font-family':'arial','marginTop': '15px'}),   #title
        
        dbc.Col([
            dbc.Row( html.H5('User inputs',style={'font-family':'arial','marginTop': '15px'})),    # subtitle
            
            # list of input widgets in first column
            dbc.Row(PNi),dbc.Row(TRi),dbc.Row(BRi),dbc.Row(NTGi),dbc.Row(WDi),dbc.Row(RRi),dbc.Row(Di),dbc.Row(Ei),dbc.Row(nui),
            
            dbc.Row(html.H6('Define data limits',style={'font-family':'arial'})), dbc.Row(ei),    # subtitle
            
            #input widgets to define the data limits
            dbc.Row(intvi, style={'margin-bottom': '10px'}),
            
            # display outputs - table of calculated variables
            dbc.Row(html.H5("Calculated variables:",style={'font-family':'arial','marginTop': '10px'})),    # subtitle
            dbc.Row(dt.DataTable(id='dt1', columns=[],style_as_list_view=True, style_cell={'textAlign': 'left','font-family': 'Arial','padding': '8px'},style_header={ 'display': 'none' ,'marginTop': '0px'},style_table={'marginTop': '0px'},
                     style_cell_conditional=[{'if': {'column_id': 'Calculated variable'},'textAlign': 'right'}]))
        ],width=4),        
        
        dbc.Col([
            # display mian outputs
            dbc.Row(html.H5('Calculated subsidence/compaction',style={'font-family':'arial','marginTop': '15px'} )),    # subtitle
            dbc.Row(html.Div(id='sub',style={'fontWeight': 'bold','font-family':'arial'})),
            dbc.Row(html.Div(id='compaction', style={'fontWeight': 'bold','font-family':'arial','marginBottom': '15px'})),
            dbc.Row(html.Div(id='strain_OB',style={'fontWeight': 'bold','font-family':'arial'})),
            dbc.Row(html.Div(id='strain_res',style={'fontWeight': 'bold','font-family':'arial','marginBottom': '15px'})),
            
            # display subsidence profile graph
            dbc.Row(dcc.Graph(id='graph')),
        ],width=4),
        
    ]),])], body=True)


# define the call back functions - list the inputs that are needed to be referenced and define outputs with IDs as defined in dash app
@callback(
    Output('dt1', 'columns'),          # calculated variables output
    Output('dt1', 'data'),             # calculated variables output
    Output('sub', 'children'),         # subsidence output
    Output('compaction', 'children'),  # compaction output
    Output('strain_res', 'children'),  # reservoir strain output
    Output('strain_OB', 'children'),   # overburden strain output
    Output('graph', 'figure'),         # Graph output
    Input(component_id='TR', component_property='value'),
    Input(component_id='BR', component_property='value'),
    Input(component_id='NTG', component_property='value'),
    Input(component_id='WD', component_property='value'),
    Input(component_id='E', component_property='value'),
    Input(component_id='nu', component_property='value'),
    Input(component_id='RR', component_property='value'),
    Input(component_id='e', component_property='value'),
    Input(component_id='intv', component_property='value'),
    Input(component_id='D', component_property='value'),
)

def appcalc(TR,BR,NTG,WD,Ew,nuw,RR,e,intv,D):
    # calcution using functions
    
    TR, BR, NTG, WD, Ew, nuw,RR, RT,OB,K,C,M,B,Cm,data,columns = calcvariables(TR,BR,NTG,WD,Ew,nuw,RR)
    df_results,sub,compaction,Compaction, sb_sub,strain_res,strain_OB = geertsma(TR, BR, NTG, WD, Ew, nuw,RR, e,intv,D)
    fig = graph(TR, BR, NTG, WD, Ew, nuw,RR, e,intv,D)

    return columns,data,sub,compaction,strain_res,strain_OB, fig, #outputs 

if __name__ == '__main__':    # run app
    app.run_server(port=8058,debug=True,jupyter_mode="external") 

Dash app running on http://127.0.0.1:8058/
